## 1. Load Data

#### 1.1 Import images

In [7]:
import os
import cv2
import numpy as np

path_img_src = os.path.join(os.getcwd(), 'data/processed')
valid_images = [".jpg",".gif",".png",".tga"]
x = []
for file in os.listdir(path_img_src):
    ext = os.path.splitext(file)[1]
    if ext.lower() not in valid_images: continue
    img = cv2.imread(os.path.join(path_img_src,file), cv2.IMREAD_GRAYSCALE)
    x.append(img)
x = np.array(x)
x = x.reshape(x.shape[0], 1, x.shape[1], x.shape[2])
x.shape

(24960, 1, 75, 100)

#### 1.2 Import labels

In [8]:
path_labels_src = os.path.join(os.getcwd(), 'data')
y = []
with open(os.path.join(path_labels_src, 'labels.txt'), "r") as labels_file:
    for line in labels_file:
        y.append(int(line.rstrip('\n')))
y = np.array(y[:x.shape[0]]).reshape(-1,1)
y.shape

(24960, 1)

## 2. Preprocess Data

#### 2.1 Normalize images

In [9]:
x = x / 255
np.min(x), np.max(x)

(0.0, 1.0)

#### 2.2 Check label bias

In [10]:
from collections import Counter
c = Counter(y.reshape(-1).tolist())
print(c)

y_orig = y

Counter({1: 16296, 0: 5288, 5: 1239, 6: 1068, 3: 464, 4: 316, 2: 249, 7: 39, 8: 1})


In [6]:
y = y_orig

if c[6]/c[5] > 1.1:
    occ = c[6] - c[5]
    idx_6 = np.array(np.where(y.reshape(-1) == 6))[0]
    idx = np.random.randint(0, c[6], occ)
    idx_6 = idx_6[idx]
    np.delete(y,idx_6)
    np.delete(x,idx_6)
elif c[6]/c[5] < 0.9:
    occ = c[6] - c[5]
    idx_5 = np.array(np.where(y.reshape(-1) == 5))[0]
    idx = np.random.randint(0, c[5], occ)
    idx_5 = idx_5[idx]
    y = np.delete(y,idx_5)
    x = np.delete(x,idx_5, axis=0)
print(Counter(y.reshape(-1).tolist()))

#check if front is equivalent to right and left
if c[1]/(c[6]+c[5]/2) > 1.3:
    occ = c[1] - int((c[6]+c[5])/2)
    idx_1 = np.array(np.where(y.reshape(-1) == 1))[0]
    idx = np.random.randint(0, c[1], occ)
    idx_1 = idx_1[idx]
    y = np.delete(y,idx_1)
    x = np.delete(x,idx_1, axis=0)
print(Counter(y.reshape(-1).tolist()))

ValueError: negative dimensions are not allowed

#### 2.3 Convert labels to one-hot encoding

In [11]:
from keras.utils import to_categorical
number_of_classes = 9
print("Before: ", y[0])
y = to_categorical(y, num_classes=number_of_classes)
print("After:  ", y[0])

Using TensorFlow backend.


Before:  [5]
After:   [ 0.  0.  0.  0.  0.  1.  0.  0.  0.]


#### 2.4 Split data for train and test

In [12]:
seed = 1
np.random.seed(seed)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=np.random)
print("Train Data | Test Data")
print(("{0:^10} | {1:^10}").format(x_train.shape[0], x_test.shape[0]))

Train Data | Test Data
  19968    |    4992   


## 3. Build Model

#### 3.1 Import Keras and use NCHW mode

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import BatchNormalization
from keras.utils import np_utils

from keras import backend
backend.set_image_dim_ordering('th')

#### 3.2 Create reusable functions

In [14]:
c = x.shape[1]
h = x.shape[2]
w = x.shape[3]
no_of_classes = y.shape[1]
print(x.shape, y.shape)

def train_model(model, epochs=100, batch_size=125):
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

def evaluate_model(model):
    print("\n\n")
    scores = model.evaluate(x_test, y_test)
    print("Accuracy: ", scores[1]*100, "%")
    return scores

(24960, 1, 75, 100) (24960, 9)


#### 3.3 Create a baseline model

In [15]:
def baseline_model():
    model = Sequential()
    
    model.add(Conv2D(32, (7, 7), kernel_initializer='normal', activation='relu', input_shape=(c, h, w)))
    model.add(Conv2D(32, (7, 7), kernel_initializer='normal', activation='relu', input_shape=(c, h, w)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    # model.add(BatchNormalization())
    
    model.add(Conv2D(64, (5, 5), kernel_initializer='normal', activation='relu'))
    model.add(Conv2D(64, (5, 5), kernel_initializer='normal', activation='relu'))
    model.add(Conv2D(64, (5, 5), kernel_initializer='normal', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    # model.add(BatchNormalization())
    
    model.add(Conv2D(128, (3, 3), kernel_initializer='normal', activation='relu'))
    model.add(Conv2D(128, (3, 3), kernel_initializer='normal', activation='relu'))
    model.add(Conv2D(128, (3, 3), kernel_initializer='normal', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    # model.add(BatchNormalization())

    # model.add(Conv2D(256, (3, 3), kernel_initializer='normal', activation='relu'))
    # model.add(Conv2D(256, (3, 3), kernel_initializer='normal', activation='relu'))
    # model.add(Conv2D(256, (3, 3), kernel_initializer='normal', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.2))
    # # model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dense(2048, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1024, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(no_of_classes, kernel_initializer='normal', activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = baseline_model()
train_model(model, 50, 125)
evaluate_model(model)

Epoch 1/50
19968/19968 [==============================] - 23s 1ms/step - loss: 1.1530 - acc: 0.6473
Epoch 2/50
19968/19968 [==============================] - 19s 955us/step - loss: 1.0885 - acc: 0.6515
Epoch 3/50
19968/19968 [==============================] - 19s 959us/step - loss: 1.0815 - acc: 0.6515
Epoch 4/50
19968/19968 [==============================] - 19s 957us/step - loss: 1.0797 - acc: 0.6515
Epoch 5/50
19968/19968 [==============================] - 19s 951us/step - loss: 1.0785 - acc: 0.6515
Epoch 6/50
19968/19968 [==============================] - 19s 949us/step - loss: 1.0721 - acc: 0.6515
Epoch 7/50
19968/19968 [==============================] - 19s 958us/step - loss: 1.0605 - acc: 0.6519
Epoch 8/50
19968/19968 [==============================] - 19s 955us/step - loss: 1.0444 - acc: 0.6558
Epoch 9/50
19968/19968 [==============================] - 19s 952us/step - loss: 1.0301 - acc: 0.6612
Epoch 10/50
19968/19968 [==============================] - 19s 957us/step - loss: 1.

[0.53564166946288871, 0.82451923076923073]

## 4. Saving and loading model architecture and weights

#### 4.1 Define reusable functions

In [16]:
import h5py
from keras.models import model_from_json
from keras.models import load_model

path_models = os.path.join(os.getcwd(), 'models')
valid_model_files = [".h5", ".json"]
weights_suffix = '_weights.h5'
architecture_suffix = '_architecture.json'

In [17]:
def get_last_file_number(path):
    numbers = [-1]
    for file in os.listdir(path):
        filename = os.path.splitext(file)[0]
        ext = os.path.splitext(file)[1]
        if ext.lower() not in valid_model_files: continue
        if filename.startswith('model_'): 
            numbers.append(int(''.join(list(filter(str.isdigit, filename)))))
    counter = max(numbers)
    return counter

#### 4.2 Save current model

In [18]:
counter = 0
if not os.path.isdir(path_models): os.makedirs(path_models)
else: counter = get_last_file_number(path_models) + 1

model_name = 'model_' + '{0:03d}'.format(counter)
model_arch_file = path_models + '\\' + model_name + architecture_suffix
model_weights_file = path_models + '\\' + model_name + weights_suffix
with open(model_arch_file, 'w+') as json_file:
    json_file.write(model.to_json(indent=4))
model.save_weights(model_weights_file)

model.save(path_models + '\\' + model_name + '.h5')

print("Saving files:\n\t" + model_name + architecture_suffix + '\n\t' + model_name + weights_suffix)

Saving files:
	model_008_architecture.json
	model_008_weights.h5


#### 4.3 Load model from file

In [21]:
counter = get_last_file_number(path_models)
    
model_name = 'model_' + '{0:03d}'.format(counter)
model_arch_file = path_models + '\\' + model_name + architecture_suffix
model_weights_file = path_models + '\\' + model_name + architecture_suffix

print("Loading files:\n\t" + model_name + architecture_suffix + '\n\t' + model_name + weights_suffix)

if not os.path.isfile(model_arch_file):
    print('Could not find', model_arch_file)
elif not os.path.isfile(model_weights_file):
    print('Could not find', model_weights_file)
else:
    # with open(model_arch_file, 'r') as json_file:
        # model_loaded = model_from_json(json_file.read())
    # model_loaded.load_weights(model_weights_file)
    model_loaded = load_model(path_models + '\\' + model_name + '.h5')

Loading files:
	model_008_architecture.json
	model_008_weights.h5


In [22]:
evaluate_model(model_loaded)




4992/4992 [==============================] - 2s 495us/step
Accuracy:  82.4519230769 %


[0.53564166946288871, 0.82451923076923073]